### Управление Prepaid пакетами

In [3]:
# %reset -f
from zeep import Client
from zeep.wsse.username import UsernameToken
import datetime as dt

connect = input("Choose connection (prod or lab):")

if connect == 'lab':
    user = 'lab_api@roamability.com'
    password = '25D55AD283AA400AF464C76D713C07AD' #Lab
    api_link = 'http://172.20.39.7:8585/ocsapi/roamability/api/roamability.wsdl' #Lab
elif connect == 'prod':
    user = 'prod_api@roamability.com'
    password = '25D55AD283AA400AF464C76D713C07AD'
    api_link = 'https://ocsapi.roamability.com:7443/ocsapi/roamability/api/roamability.wsdl'

user_name_token = UsernameToken(user, password)
user_name_token.use_digest = True
client = Client(api_link, wsse=user_name_token)

reseller = client.service.getResellerInfo()
accounts = client.service.getAccounts()

if reseller.result.code == '1':
    print('You are connected to Reseller: {}.\
    \nAvailable credit for the Reseller: {:.0f} USD.\n'.\
    format(reseller.reseller.resellerName,reseller.reseller.availableCredit))
else:
    print('Error in getting Reseller info: {}'.format(reseller.result.description))

if accounts.result.code == '1':
    accounts_dict = {account.accountId:account.accountName for account in accounts.accounts.accounts}
    print('The Reseller contains the following accounts:')
    for i,[account_id,account_name] in enumerate(accounts_dict.items()):
        print(' {:<5}{} (accountID: {})'.format(i,account_name,account_id))
else:
    print('Error in getting Account info: {}'.format(accounts.result.description))

Choose connection (prod or lab):lab
You are connected to Reseller: Roamability SIMCARDS Invertory.    
Available credit for the Reseller: 99 USD.

The Reseller contains the following accounts:
 0    Roamability Inventory (accountID: 352358)


In [13]:
#######################################################################################
# Define variables here
#######################################################################################

imsi_list = range(42501961302149, 42501961302150 + 1)
activation_date = dt.datetime.strptime('2020-09-11','%Y-%m-%d') # activation_date + dt.timedelta(days=143)
prepaid_template_id = 1349

#######################################################################################

#######################################################################################

In [14]:
# Run the scripr

for i, imsi in enumerate(imsi_list):
    subs = client.service.getSubscriberByIMSI(imsi)
    if subs.result.code == '1':
        pp_ass = client.service.assignSubscriberPrePaidPackage(subs.subscriber.subscriberId,
                                                               prepaid_template_id, activation_date)
        if pp_ass.result.code == '1':
            print(f"""{imsi}: {pp_ass.result.description}; \
LZ: {pp_ass.prePaidPackage.locationZone.locationZoneName}; \
DZ: {pp_ass.prePaidPackage.destinationZone.destinationZoneName}; \
{dt.datetime.strftime(pp_ass.prePaidPackage.activationDate,'%Y-%m-%d')} - \
{dt.datetime.strftime(pp_ass.prePaidPackage.expirationDate,'%Y-%m-%d')}""")
        else:
            print(f"""{imsi}: {pp_ass.result.description}""")
            
    else:
        print(f"""{imsi}: {subs.result.description}""")

42501961302149: Subscriber not found
42501961302150: Subscriber not found


### Developments

In [ ]:
# Subscriber in lab in STI account
#subscriber_id = client.service.getSubscriberById('934420')
#subscriber_iccid = client.service.getSubscriberByICCID('8997219121000031446')
# client.service.getSubscriberPrePaidPackages('831882', 1)
#client.service.getSubscriberByIMSI('425019613023160')
#client.service.getSubscriberById('209287')
#client.service.getSubscriberByICCID('8997219121000022660')
# client.service.assignSubscriberPrePaidPackage('831882', 1, dt.datetime.strptime('2019-07-07','%Y-%m-%d'))
# client.service.getSubscriberPrePaidPackages('937939', 1)
# client.service.expireSubscriberPrePaidPackage(1848972,60311)
# client.service.getSubscriberPrePaidPackages(1848972,1)
# prices = client.service.getMobilePrices() # prices.destinations[0]
# client.service.getPricingPlans()
# client.service.modifySubscriberSpeedLimit(1449606, '256kbps')
# client.service.modifySubscriberSpeedLimit(210000, '1024kbps') # lab
#subscriber_id